# 😸 <b>공겹춘배</b>

알리오
https://alio.go.kr/

잡알리오
https://job.alio.go.kr/

잡플래닛
https://www.jobplanet.co.kr/

## 📚 <b>라이브러리</b>

In [ ]:
from bs4 import BeautifulSoup
import requests
import time
import re

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import platform

# from selenium import webdriver
# from selenium.webdriver import ActionChains
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.common.by import By

#한글 폰트
if platform.system()== 'Windows' :
    plt.rc('font', family='NanumGothic')
else:
    plt.rc('font', family = 'AppleGothic')

## 🛠 <b>알리오 공공기관 코드 크롤링</b>


알리오 370여개 공공기관 스크립트 작동 내부 url 코드 크롤링


알리오 - [항목 별 공시] - [직원 평균보수 현황] 정보는 문서 스크립트로 동작, 바로 크롤링 할 수 없음.


각 공공기관 별 내부 url 코드에서 규칙을 찾아 크롤링 (C0001 ~ C1370)


+ https://alio.go.kr/item/itemReportTerm.do?apbaId=C0001&reportFormRootNo=20601#toc-124


script 에 disclosureNo('2021041302190478') 형식의 공개 번호 발견, html 문서에 직접 접근해 크롤링 할 수 있게 됨.


+ https://alio.go.kr/upload/disclosure/2021/04/13/2021041302190478/doc.html

In [ ]:
# 알리오 372개 공공기관 내부 url 코드 크롤링 (7분 40초 걸림)

codedict = {"institution":[], "code":[]}

for i in range(1, 1370):
    
    if i < 10 :
        urlcode = 'C000{}'.format(i)
    elif 10 <= i | i < 100 :
        urlcode = 'C00{}'.format(i)
    elif 100 <= i | i < 1000 :
        urlcode = 'C0{}'.format(i)
    elif 1000 <= i | i < 10000 :
        urlcode = 'C{}'.format(i)

    url = 'https://alio.go.kr/item/itemReportTerm.do?apbaId={}&reportFormRootNo=20601'.format(urlcode)

    response = requests.get(url)
    response.raise_for_status()

    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        #print(response.status_code)
    else :
        print(response.status_code)

    institution = soup.select_one('#p_header_wrap > div.hl_area > div > div > span')

    if institution != None:
        codedict['institution'].append(institution.text)
        codedict['code'].append(urlcode)
        print('institution: ' + institution.text + ', code: ' + urlcode)

    i+=1


### 추출한 url 코드 저장

In [ ]:
df = pd.DataFrame(codedict)
df

# 한국광해광업공단(2021.12.08 지정) 이전 데이터 없음 -> 삭제
# **한국율도 데이터 없음 -> 삭제
df.drop(index=226, axis=0, inplace=True)
df.drop(index=371, axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)

# csv 파일로 저장
df.to_csv('institution_code.csv', index=False)

# 제대로 저장되었는지 확인
institution_code = pd.read_csv('institution_code.csv')
institution_code

In [ ]:
institution_code = pd.read_csv('institution_code.csv')
institution_code
institution_code[institution_code['institution']=='**한국율도']

institution_code.drop(index=226, axis=0, inplace=True)
institution_code.drop(index=371, axis=0, inplace=True)
institution_code.reset_index(drop=True, inplace=True)
institution_code

## 🛠 <b>알리오 정보 크롤링</b>
disclosureNo('2021041302190478') 형식의 공개 번호를 통해 알리오 html 문서 크롤링 

https://alio.go.kr/upload/disclosure/2021/04/13/2021041302190478/doc.html

+ 신입사원 초임 (new_sal)
+ 직원 평균연봉 (avg_sal)
+ 평균 근속연수 (y_service)
>각각 6년치 데이터 크롤링(2016, 2017, 2018, 2019, 2020, 2021)

In [ ]:
keyword = '국립해양과학관'
urlword = 'C0003'
#print(urlword) # C1333

url = 'https://alio.go.kr/item/itemReportTerm.do?apbaId={}&reportFormRootNo=20601'.format(urlword)

response = requests.get(url)
response.raise_for_status()

if response.status_code == 200:
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    #print(response.status_code)
else :
    print(response.status_code)


script = soup.select("body > script:nth-child(4)")
str(script[0])
regex=re.compile("disclosureNo:\"(\d+)\"")
disclosureNo = regex.findall(str(script[0]))[0]
disclosureyear = disclosureNo[:4]
disclosuremonth = disclosureNo[4:6]
disclosureday = disclosureNo[6:8]

url = 'https://alio.go.kr/upload/disclosure/{}/{}/{}/{}/doc.html'.format(disclosureyear, disclosuremonth, disclosureday, disclosureNo)

response = requests.get(url)
response.raise_for_status()

if response.status_code == 200:
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    #print(response.status_code)
else :
    print(response.status_code)

address = soup.find(text="1인당 평균 보수액")
neu = address.parent
neu = neu.parent
print(neu.text.strip().split('\n'))

address = soup.find(text="평균근속연수(개월)")
neu2 = address.parent
neu2 = neu2.parent
print(neu2.text.strip().split('\n'))

address = soup.find(text="합계")
neu3 = address.parent
neu3 = neu3.parent
print(neu3.text.strip().split('\n'))


In [ ]:
class GetAlioInfo :
    
    def __init__(self, keyword) :
        urlword = institution_code[institution_code['institution']==keyword]['code'].values[0]

        url = 'https://alio.go.kr/item/itemReportTerm.do?apbaId={}&reportFormRootNo=20601'.format(urlword)

        response = requests.get(url)
        response.raise_for_status()

        html = response.text
        self.soup = BeautifulSoup(html, 'html.parser')

        script = self.soup.select("body > script:nth-child(4)")
        str(script[0])
        regex=re.compile("disclosureNo:\"(\d+)\"")
        disclosureNo = regex.findall(str(script[0]))[0]
        disclosureyear = disclosureNo[:4]
        disclosuremonth = disclosureNo[4:6]
        disclosureday = disclosureNo[6:8]

        url = 'https://alio.go.kr/upload/disclosure/{}/{}/{}/{}/doc.html'.format(disclosureyear, disclosuremonth, disclosureday, disclosureNo)

        response = requests.get(url)
        response.raise_for_status()

        if response.status_code == 200:
            html = response.text
            self.soup = BeautifulSoup(html, 'html.parser')
            #print(response.status_code)
        else :
            print(response.status_code)
    
    def get_new_sal(self) :
        address = self.soup.find("a", text="2. 신입사원 초임").findNext(text="합계").parent.parent
        new_sal = address.text.strip().split('\n')
        new_sal[0] = '신입사원 초봉'
        return(new_sal)

    def get_avg_sal(self) :
        address = self.soup.find("td", text="정규직(일반정규직)").findNext(text="1인당 평균 보수액").parent.parent
        avg_sal = address.text.strip().split('\n')
        return(avg_sal)

    def get_y_service(self) :
        address = self.soup.find("td", text="정규직(일반정규직)").findNext(text="평균근속연수(개월)").parent.parent
        y_service = address.text.strip().split('\n')
        return(y_service)


### 알리오 직원 평균연봉 크롤링+저장
직원 평균연봉(avg_sal) -> alio_avg_sal.csv

In [ ]:
avg_sal_dict = {"institution":[], "avg16":[], "avg17":[], "avg18":[], "avg19":[], "avg20":[], "avg21":[]}

i = 1
for keyword in institution_code['institution']:
    a = GetAlioInfo(keyword)
    templist = a.get_avg_sal()
    avg_sal_dict['institution'].append(keyword)
    avg_sal_dict['avg16'].append(templist[1].strip().replace(',',''))
    avg_sal_dict['avg17'].append(templist[2].strip().replace(',',''))
    avg_sal_dict['avg18'].append(templist[3].strip().replace(',',''))
    avg_sal_dict['avg19'].append(templist[4].strip().replace(',',''))
    avg_sal_dict['avg20'].append(templist[5].strip().replace(',',''))
    avg_sal_dict['avg21'].append(templist[6].strip().replace(',',''))
    percentage = round(i / (len(institution_code['institution'])+1) * 100, 2)
    print(percentage,'% ', keyword, templist[1:6])
    i += 1

df = pd.DataFrame(avg_sal_dict)
df.to_csv('alio_avg_sal.csv')
df

### 알리오 신입사원 초봉 크롤링+저장
신입사원 초봉(new_sal) -> alio_new_sal.csv

In [ ]:
new_sal_dict = {"institution":[], "new16":[], "new17":[], "new18":[], "new19":[], "new20":[], "new21":[]}

i = 1
for keyword in institution_code['institution']:
    a = GetAlioInfo(keyword)
    templist = a.get_new_sal()
    new_sal_dict['institution'].append(keyword)
    new_sal_dict['new16'].append(templist[1].strip().replace(',',''))
    new_sal_dict['new17'].append(templist[2].strip().replace(',',''))
    new_sal_dict['new18'].append(templist[3].strip().replace(',',''))
    new_sal_dict['new19'].append(templist[4].strip().replace(',',''))
    new_sal_dict['new20'].append(templist[5].strip().replace(',',''))
    new_sal_dict['new21'].append(templist[6].strip().replace(',',''))
    percentage = round(i / (len(institution_code['institution'])+1) * 100, 2)
    print(percentage,'% ', keyword, templist[1:6])
    i += 1
    
df = pd.DataFrame(new_sal_dict)
df.to_csv('alio_new_sal.csv')
df

### 알리오 평균 근속년수 크롤링+저장
평균 근속년수(y_service) -> alio_y_service.csv

In [ ]:
y_service_dict = {"institution":[], "ser16":[], "ser17":[], "ser18":[], "ser19":[], "ser20":[], "ser21":[]}

i = 1
for keyword in institution_code['institution']:
    a = GetAlioInfo(keyword)
    templist = a.get_y_service()
    y_service_dict['institution'].append(keyword)
    y_service_dict['ser16'].append(templist[1].strip().replace(',',''))
    y_service_dict['ser17'].append(templist[2].strip().replace(',',''))
    y_service_dict['ser18'].append(templist[3].strip().replace(',',''))
    y_service_dict['ser19'].append(templist[4].strip().replace(',',''))
    y_service_dict['ser20'].append(templist[5].strip().replace(',',''))
    y_service_dict['ser21'].append(templist[6].strip().replace(',',''))
    percentage = round(i / (len(institution_code['institution'])+1) * 100, 2)
    print(percentage,'% ', keyword, templist[1:6])
    i += 1

df = pd.DataFrame(y_service_dict)
df.to_csv('alio_y_service.csv')
df
    

## 🛠<b> 잡알리오 </b>

https://job.alio.go.kr/recruit.do

공공기관 채용 정보 크롤링

In [ ]:
dictionary = {"institution":[], "title":[], "region":[], "type":[], "start_date":[], "end_date":[], "new_sal":[], "avg_sal":[], "rate":[]}

def current_employment():

    classification = input("""
    직종을 선택해 주세요: 
    1. 경영사무,     2. 서비스 ,    3. ICT,    4. 교육/연구,    5. 산업,    6. 전부""")
    if classification == '경영사무':
        classification = '&detail_code=R600001&detail_code=R600002&detail_code=R600003'
    elif classification == '서비스':
        classification = '&detail_code=R600005&detail_code=R600006&detail_code=R600007&detail_code=R600008&detail_code=R600009&detail_code=R600010&detail_code=R600011&detail_code=R600012&detail_code=R600013'
    elif classification == 'ICT':
        classification = '&detail_code=R600019&detail_code=R600020'
    elif classification == '교육/연구':
        classification = '&detail_code=R600004&detail_code=R600025'
    elif classification == '산업':
        classification = '&detail_code=R600014&detail_code=R600015&detail_code=R600016&detail_code=R600017&detail_code=R600018&detail_code=R600021&detail_code=R600022&detail_code=R600023&detail_code=R600024'
    elif classification == '전부':
        classification = ''
    
    page = 1

    while True:

        url = 'https://job.alio.go.kr/recruit.do?pageNo={}&idx=&recruitYear=&recruitMonth={}&work_type=R1010&work_type=R1070&s_date=2021.12.08&e_date=2022.02.08&org_type=&org_name=&ing=2&title=&order=REG_DATE#'.format(page, classification)

        response = requests.get(url)
        response.raise_for_status()

        if response.status_code == 200:
            html = response.text
            soup = BeautifulSoup(html, 'html.parser')
            #print(response.status_code)
        else :
            print(response.status_code)

        tbody = soup.select_one('#frm > table > tbody')
        trs = tbody.select('tr')
        
        if len(tbody.text.strip()) == 0:
            #print('neu~')  # 내용 없는 페이지까지 출력 후 확인용
            break

        for tr in trs:
            tds = tr.select("td")
            dictionary["institution"].append(tds[3].text.replace('재단법인', '(재)').strip())
            dictionary["title"].append(tds[2].text.replace('\t', '').replace('\r', '').replace('\n', '').strip())
            dictionary["region"].append(tds[4].text.replace('\t', '').replace('\r', '').replace('\n', '').strip())
            dictionary["type"].append(tds[5].text.replace('\t', '').replace('\r', '').replace('\n', '').strip())
            dictionary["start_date"].append(tds[6].text)
            dictionary["end_date"].append(tds[7].text)

            # JobPlanet 리뷰
            try:
                keyword = tds[3].text.replace('재단법인', '(재)').strip()
                url = 'https://www.jobplanet.co.kr/search?_rs_act=search_history&_rs_con=seach&category=search_new&query={}'.format(keyword)

                response = requests.get(url)

                response.raise_for_status()
                if response.status_code == 200:
                    html = response.text
                    soup = BeautifulSoup(html, 'html.parser')
                    #print(response.status_code)
                else :
                    print(response.status_code)

                result_card = soup.select_one('#mainContents > div:nth-child(1) > div > div.result_company_card > div.is_company_card > div:nth-child(1)')
                rate = result_card.select_one('#mainContents > div:nth-child(1) > div > div.result_company_card > div.is_company_card > div:nth-child(1) > span.rate_ty02')
                dictionary['rate'].append(float(rate.text.strip()))
                print(rate.text)
            except:
                dictionary['rate'].append(None)
        page += 1

In [ ]:
current_employment()

In [ ]:
df = pd.DataFrame(dictionary)
df

In [ ]:
dfrate = pd.concat([df['institution'], df['rate'], df['new_sal'],df['avg_sal']], axis=1)
dfrate.drop_duplicates(inplace=True)
dfrate.dropna(inplace=True)
dfrate_top5 = dfrate.sort_values(['rate'], ascending=False).head(5)
dfrate_top5

In [ ]:
import matplotlib.pyplot as plt
plt.rc('font', size=15)
plt.figure(figsize=(20, 10))

x = np.arange(len(dfrate_top5['institution']))

plt.bar(x-0, dfrate_top5['rate']*10000, label='rate', color = 'b', width=0.2)
plt.bar(x+0.2, dfrate_top5['new_sal'], label='new_sal', color = 'r', width=0.2)
plt.bar(x+0.4, dfrate_top5['avg_sal'], label='avg_sal', color = 'g', width=0.2)
plt.xticks(x, dfrate_top5['institution'])
plt.legend()
plt.show()

In [ ]:
all_dictionary = {"institution":[], "title":[], "region":[], "type":[], "start_date":[], "end_date":[]}

def get_all_employment():

    classification = input("""
    직종을 선택해 주세요:
    1. 경영사무
    2. 서비스
    3. ICT
    4. 교육/연구 """)
    if classification == '경영사무':
        classification = '&detail_code=R600001&detail_code=R600002&detail_code=R600003'
    elif classification == '서비스':
        classification = '&detail_code=R600005&detail_code=R600006&detail_code=R600007&detail_code=R600008&detail_code=R600009&detail_code=R600010&detail_code=R600011&detail_code=R600012&detail_code=R600013'
    elif classification == 'ICT':
        classification = '&detail_code=R600019&detail_code=R600020'
    elif classification == '교육/연구':
        classification = '&detail_code=R600004&detail_code=R600025'
    elif classification == '산업':
        classification = '&detail_code=R600014&detail_code=R600015&detail_code=R600016&detail_code=R600017&detail_code=R600018&detail_code=R600021&detail_code=R600022&detail_code=R600023&detail_code=R600024'
    page = 1

    while True:

        url = 'https://job.alio.go.kr/recruit.do?pageNo={}&idx=&recruitYear=&recruitMonth={}&work_type=R1010&work_type=R1070&s_date=2016.01.01&e_date=2022.02.08&org_type=&org_name=&title=&order=REG_DATE#'.format(page, classification)

        response = requests.get(url)
        response.raise_for_status()

        if response.status_code == 200:
            html = response.text
            soup = BeautifulSoup(html, 'html.parser')
            #print(response.status_code)
        else :
            print(response.status_code)

        tbody = soup.select_one('#frm > table > tbody')
        trs = tbody.select('tr')
        
        if len(tbody.text.strip()) == 0:
            print('neu~')  # 내용 없는 페이지까지 출력 후 확인용
            break

        for tr in trs:
            tds = tr.select("td")
            all_dictionary["institution"].append(tds[3].text.replace('재단법인', '(재)').strip())
            all_dictionary["title"].append(tds[2].text.replace('\t', '').replace('\r', '').replace('\n', '').strip())
            all_dictionary["region"].append(tds[4].text.replace('\t', '').replace('\r', '').replace('\n', '').strip())
            all_dictionary["type"].append(tds[5].text.replace('\t', '').replace('\r', '').replace('\n', '').strip())
            all_dictionary["start_date"].append(tds[6].text)
            all_dictionary["end_date"].append(tds[7].text)

        page += 1


In [ ]:
get_all_employment()

In [ ]:
df = pd.DataFrame(all_dictionary)
df

In [ ]:
df['start_date'] = pd.to_datetime(df['start_date'], yearfirst= True)
df['end_date'] = pd.to_datetime(df['end_date'], yearfirst= True)

In [ ]:
df

In [ ]:
month = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
monthlist = []


for i in range(1, 13):
    monthlist.append(df[df['end_date'].dt.month==i]['institution'].count())

In [ ]:
monthlist

In [ ]:
import matplotlib.pyplot as plt

import pandas as pd



plt.bar(month, monthlist)

plt.show()

